In [1]:
import sys
print("Python Version:", sys.version)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
pd.set_option('display.max_rows', 500)
%matplotlib inline

Python Version: 3.8.3 (default, Jul  2 2020, 11:26:31) 
[Clang 10.0.0 ]


In [24]:
# yymmdd format
dates = ['190803',
        '190810',
        '190817',
        '190824',
        '190831',
        '190907',
        '190914',
        '190921',
        '190928']

# iterate through dates and pull in csv and cat dataframes together
df_turns = []
for date in dates:
    df_turns.append(pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_'+date+'.txt'))
    
# Concat all dataframes
df_turns = pd.concat(df_turns)  

In [25]:
#Clean up the column Names
new_col_names = [name.strip() for name in df_turns.columns]
df_turns.columns = new_col_names



In [26]:
# Remove audited rows
df_turns=df_turns[df_turns.DESC!='RECOVR AUD']
# remove column?
del df_turns['DESC']

In [27]:
# Create a DateTime column that can be used for timeseries
df_turns.loc[:,'DATETIME'] = pd.to_datetime(df_turns['DATE'] + ' ' + df_turns['TIME'])

#Convert Date column to proper date type.
df_turns['DATE']  = pd.to_datetime(df_turns['DATE'],format='%m/%d/%Y')

#convert time to datetime type
df_turns['TIME']=pd.to_datetime(df_turns['TIME'], format='%H:%M:%S')

In [28]:
#backup
df_backup=df_turns.copy()

In [29]:
#Sort in order of datetime
df_turns = df_turns.sort_values(by=['DATETIME']).reset_index()

In [30]:
# Caluclate Entries Aggregated at station level by DateTime
df_turns['Entry_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['ENTRIES'].transform(pd.Series.diff)['ENTRIES']

# Caluclate Exits Aggregated at station level by DateTime
df_turns['Exit_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['EXITS'].transform(pd.Series.diff)['EXITS']



In [31]:
#Absolute Value to deal with counting backward issues 
df_turns['Entry_Diff'] = abs(df_turns['Entry_Diff'])
df_turns['Exit_Diff']=abs(df_turns['Exit_Diff'])

# Calculate both
df_turns['Total_Traffic']=df_turns['Entry_Diff']+df_turns['Exit_Diff']

In [34]:
# I think we can use this as a cut off point for station traffic totals.
#df_turns.groupby(['STATION'])['Total_Traffic'].quantile(.99)
df_turns.sample()

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,Entry_Diff,Exit_Diff,Total_Traffic
1137499,79520,N309A,R140,00-02-00,QUEENS PLAZA,EMR,IND,2019-09-03,1900-01-01 17:00:00,51379644,17219520,2019-09-03 17:00:00,280.0,77.0,357.0


### ==================================================================
### Rami's Input where I remove the Outliers

In [36]:
def Add_Weekday(data_frame, column='Date'):
    dmap = {0:'Mon', 1: 'Tue', 2: 'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
    #Convert column to date
    data_frame[column] = pd.to_datetime(data_frame[column])
    #Add column which shows the Weekday in integer
    data_frame['Day_Number'] = data_frame[column].apply(lambda x: x.dayofweek)
    #Add column which shows the Weekday in words
    data_frame['Weekday'] = data_frame['Day_Number'].map(dmap)
    return data_frame
df_turns_backup2 = df_turns.copy()

In [37]:
df_turns = Add_Weekday(df_turns, 'DATE')


,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,Entry_Diff,Exit_Diff,Total_Traffic,Day_Number,Weekday
0,0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-07-27,1900-01-01,7148288,2418935,2019-07-27,NaN,NaN,NaN,5,Sat
1,112391,N605,R024,00-05-01,SUTPHIN-ARCHER,EJZ,IND,2019-07-27,1900-01-01,0,1168,2019-07-27,NaN,NaN,NaN,5,Sat
2,112349,N605,R024,00-05-00,SUTPHIN-ARCHER,EJZ,IND,2019-07-27,1900-01-01,33556743,16777216,2019-07-27,NaN,NaN,NaN,5,Sat
3,198692,R612,R057,01-03-03,ATL AV-BARCLAY,BDNQR2345,IRT,2019-07-27,1900-01-01,2120496,1957954,2019-07-27,NaN,NaN,NaN,5,Sat
4,112307,N605,R024,00-00-06,SUTPHIN-ARCHER,EJZ,IND,2019-07-27,1900-01-01,5050187,2146519,2019-07-27,NaN,NaN,NaN,5,Sat


In [38]:
#Filter out the Null Values
df_turns.dropna(subset=["Entry_Diff","Exit_Diff",'Total_Traffic'], axis=0, inplace=True)
df_turns.head()

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,Entry_Diff,Exit_Diff,Total_Traffic,Day_Number,Weekday
2554,118307,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:24:57,65,30,2019-07-27 00:24:57,2.0,0.0,2.0,5,Sat
2592,118308,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:38:57,65,30,2019-07-27 00:38:57,0.0,0.0,0.0,5,Sat
2636,118309,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:52:57,65,30,2019-07-27 00:52:57,0.0,0.0,0.0,5,Sat
2640,118310,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:56:25,4043,3513,2019-07-27 00:56:25,3978.0,3483.0,7461.0,5,Sat
4489,118311,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 01:06:57,65,30,2019-07-27 01:06:57,3978.0,3483.0,7461.0,5,Sat


In [39]:
dfc_grouped = df_turns.groupby(['DATE', 'STATION', 'Weekday'])['Entry_Diff', 'Exit_Diff', 'Total_Traffic'].sum().reset_index()

<ipython-input-39-41ea38c374a6>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dfc_grouped = df_turns.groupby(['DATE', 'STATION', 'Weekday'])['Entry_Diff', 'Exit_Diff', 'Total_Traffic'].sum().reset_index()


In [46]:
#The upper_quant_df outputs a dataframe of the upper 95% values for each station
upper_quant_df = dfc_grouped.groupby(['STATION'])['Total_Traffic'].quantile([.95]).reset_index()
#Then I merge it with the grouped data 
df_turns_no_outlier = dfc_grouped.merge(upper_quant_df, on='STATION')
df_turns_no_outlier.head()

,DATE,STATION,Weekday,Entry_Diff,Exit_Diff,Total_Traffic_x,level_1,Total_Traffic_y
0,2019-07-27,1 AV,Sat,4308.0,5146.0,9454.0,0.95,40228.0
1,2019-07-28,1 AV,Sun,4106.0,5309.0,9415.0,0.95,40228.0
2,2019-07-29,1 AV,Mon,14483.0,16525.0,31008.0,0.95,40228.0
3,2019-07-30,1 AV,Tue,16375.0,18206.0,34581.0,0.95,40228.0
4,2019-07-31,1 AV,Wed,16812.0,19777.0,36589.0,0.95,40228.0


In [49]:
#delete the rows where The difference between Total_Traffic_y and Total_Traffic_x < 0
df_turns_no_outlier['Diff'] = df_turns_no_outlier['Total_Traffic_y'] - df_turns_no_outlier['Total_Traffic_x']
df_turns_no_outlier = df_turns_no_outlier[df_turns_no_outlier['Diff']>= 0]

In [50]:
df_turns_no_outlier.sample(50)

,DATE,STATION,Weekday,Entry_Diff,Exit_Diff,Total_Traffic_x,level_1,Total_Traffic_y,Diff
7570,2019-08-12,AQUEDUCT N.COND,Mon,846.0,1586.0,2432.0,0.95,2987.3,555.3
19902,2019-08-04,PELHAM BAY PARK,Sun,4068.0,2902.0,6970.0,0.95,12351.7,5381.7
20216,2019-08-03,PROSPECT AV,Sat,6842.0,4687.0,11529.0,0.95,22427.4,10898.4
13114,2019-08-13,EASTN PKWY-MUSM,Tue,3024.0,3166.0,6190.0,0.95,8391.2,2201.2
218,2019-08-25,104 ST,Sun,742.0,334.0,1076.0,0.95,7027.6,5951.6
2491,2019-08-30,20 AV,Fri,8243.0,4277.0,12520.0,0.95,15539.4,3019.4
21630,2019-08-31,STEINWAY ST,Sat,6533.0,6126.0,12659.0,0.95,26987.0,14328.0
11961,2019-09-26,CONEY IS-STILLW,Thu,13859.0,12188.0,26047.0,0.95,36632.0,10585.0
14660,2019-09-16,GRD CNTRL-42 ST,Mon,143003.0,147084.0,290087.0,0.95,2310450.7,2020363.7
19789,2019-08-17,PATH WTC 2,Sat,1312.0,3302.0,4614.0,0.95,17942.2,13328.2


### ==================================================================


In [77]:
# Filter out values in calculated fields outside of upper quartile range
df_turns.dropna(subset=["Entry_Diff","Exit_Diff",'Total_Traffic'], axis=0, inplace=True)

q_9 = df_turns['Total_Traffic'].quantile(.99)

df_turns_filt = df_turns[df_turns.Total_Traffic<= q_9]



In [78]:
df_per_station = df_turns_filt.groupby(['STATION']).sum()
df_per_station.sort_values('Total_Traffic', ascending=False).head(100)

,index,ENTRIES,EXITS,Entry_Diff,Exit_Diff,Total_Traffic
STATION,,,,,,
34 ST-PENN STA,3401972217,1460365699649,1538055094977,8254361.0,6992507.0,15246868.0
GRD CNTRL-42 ST,3752729625,919546458460,1105055715021,5679047.0,4802427.0,10481474.0
23 ST,2607575657,2091252915203,1812031229789,5750528.0,4288698.0,10039226.0
FULTON ST,3245309619,1787737426085,1745103246838,5078558.0,4419547.0,9498105.0
34 ST-HERALD SQ,1402968886,1646402859827,2064647805215,4784950.0,4558398.0,9343348.0
TIMES SQ-42 ST,2162196120,2135387457561,1652034183983,4821028.0,4298798.0,9119826.0
42 ST-PORT AUTH,975287980,2571982579745,2230353121601,5268731.0,3669247.0,8937978.0
14 ST-UNION SQ,787362551,88115964177,57426714905,3903559.0,3995590.0,7899149.0
86 ST,2581504936,599888099997,877774250646,4261140.0,3569073.0,7830213.0
